# Imports

In [ ]:
# Standard python library-imports
import os
import dill as pickle
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# directory
notebook_dir = os.getcwd()
current_dir ='../../../../../'
os.chdir(current_dir)
os.getcwd()

In [2]:
from src.codes.basic import *
from src.codes.utils.fem_utils_StrucMech import *
from src.codes.utils.plot_files.plot_utils import OneDPlot as plot
from src.codes.utils.plot_files.plot_utils import data_stats
from src.codes.utils.rom_utils import *
from src.codes.utils.dynamical_systems import *
import time as time

from src.codes.prob_classes.structural_mechanics.base_class_struc_mech_continuous_vibration import ROM_simulation_p
from src.codes.algorithms.ecsw import ecsw_red
from src.codes.algorithms.ecsw import ecsw_red_SS_parametric


---

### Epistemic Uncertainty[Number of snapshots]

In [3]:
Model_error_d = []
Model_error_v = []

Model_error_ROM_d = []
Model_error_ROM_v = []

### Train-Test split

In [4]:
# NLS_test = np.load(notebook_dir+'/data/test_data.npy')
# param_list_test = np.load(notebook_dir+'/data/param_list_test.npy')

##### Parameters

In [5]:
T = 2 / (2 * np.pi)  # Period of the vibration
s_param = (-2**(-0.5)+0.5, 0.0) # For F
tau_param = (T/20, T/2) # For K

param_ranges = [tau_param, s_param]
method = 'sobol'

In [6]:
p=3
# N_snap = 2

In [ ]:
p+=1
N_snap=2**p
print(N_snap)

### Generate Training datasets using different distribution

In [ ]:
os.getcwd()

In [ ]:
import numpy as np
from examples.structural_mechanics.Transverse.continuous_vibrations.oneD_beam.data_gen_multiparam import data_gen_multiparam
# from tests_.UQ.VIBRATION.oneD_beam.data_gen_UQ import data_gen_multiparam


# from src.codes.prob_classes.structural_mechanics.base_class_struc_mech_NL_static_axial import ROM_simulation_UQ

# Step 2: Modify the parameter
# Specify mean and standard deviation

# filename_dataC = f'data/DataClass_UQ_DYNAMIC_NL_{N_snap}_{method}.dill'
filename_dataC = f'data/Test_params_16_tau_s_low_mat_damping_{N_snap}_{method}.dill'

In [9]:
# filename_dataC = f'data/training_data.dill'
# generate = True

if os.path.exists(notebook_dir+'/'+filename_dataC):

    generate = False

    with open(notebook_dir+'/'+filename_dataC, 'rb') as f:
        Data_cls = pickle.load(f)

else:
    
    generate = True
    params = generate_sobol(len(param_ranges), N_snap, param_ranges)
    print(params)

# generate_gaussian_samples
# generate_sobol

In [10]:
if(generate):
    tic = time.time()
    Data_cls = UQ_data_gen(params,filename_dataC)
    toc = time.time()
    fom_time = toc-tic

In [13]:
FOS = Data_cls.FOS
d = FOS.data
mask_dv = np.append(d.mask, d.mask)

param_list = np.asarray(Data_cls.param_list)
NLS = np.asarray(Data_cls.NL_solutions)

K_mus = Data_cls.K_mus
C_mus = Data_cls.C_mus
t = Data_cls.t
q_mus = Data_cls.q_mus
T = Data_cls.T

In [ ]:
## Details of the data
print(f"Total number of datasets {len(NLS)}")

N_params = NLS.shape[0]
N_dim = NLS.shape[1]
nt = NLS.shape[2]

NLS.shape

---

#### Visualize clustered data

In [ ]:
ntd = 200+1
for i in range(clusters.max()):

    fig,ax = plt.subplots(figsize=(5,3))
    data_col = 150
    # len(NLS)
    for j in range(0,len(NLS_cl[i])):
        solution_snapshot_f_nset0 = NLS_cl[i][j][:,nt-ntd:nt]
        print(solution_snapshot_f_nset0.shape)
        ax.plot(t[-ntd:], solution_snapshot_f_nset0[data_col,:])

    plt.xlabel("Time")
    plt.ylabel("Magnitude")
    plt.show()

# Process Data

##### DOF selection

In [21]:
# # Assuming 'original_array' has shape (9, 404, 5001)
# # Transpose to switch dimensions to (9, 5001, 404), preparing for the vertical stack
# Data=NLS_cl.transpose(0, 2, 1)

# # Reshape to stack vertically, which now simply flattens the first two dimensions
# solution_snapshot_orig = Data.reshape(-1, Data.shape[-1])

cluster = 0

solution_snapshot_orig = NLS_cl[cluster][0].T

N_params = len(NLS_cl[cluster])
N_dim = solution_snapshot_orig.shape[1]
nt = solution_snapshot_orig.shape[0]

for i in range(1,N_params):
    solution_snapshot_orig = np.vstack([solution_snapshot_orig,NLS_cl[cluster][i].T])


solution_snapshot_orig = solution_snapshot_orig[:,mask_dv]

##### Size of the data

In [ ]:
solution_snapshot_orig.shape # (N_params x nt) x n_dim

#### Visualize

In [ ]:
fig,ax = plt.subplots(figsize=(5,3))
data_col = 350
# len(NLS)
for i in range(1,N_params):
    solution_snapshot_f_nset0 = solution_snapshot_orig[(i)*(nt)-ntd:(i)*(nt),:]
    ax.plot(t[-ntd:], solution_snapshot_f_nset0[:,data_col])

plt.xlabel("Time")
plt.ylabel("Magnitude")
plt.show()

In [24]:
#  solution_snapshot_f_nset0[:,180]

---

##### Temporal processing

In [25]:
solution_snapshot_f, nt, fs, tstop, ndata, nset, tn, q_mus_truncated = fom_data_processor_stacked_q(t, T, solution_snapshot_orig, q_mus_cl[cluster])

##### Size

In [26]:
# q_mus_truncated[0][0].shape

#### Visualize

**A**

In [ ]:
fig,ax = plt.subplots(figsize=(5,3))
# len(NLS)
solution_snapshot_f_nset=np.zeros((N_params,nt,solution_snapshot_f.shape[-1]))

for i in range(N_params):
    solution_snapshot_f_nset[i] = solution_snapshot_f[i*nt:(i+1)*nt,:]
    ax.plot(tn, solution_snapshot_f_nset[i][:,380])

plt.xlabel("Time")
plt.ylabel("Magnitude")
plt.show()


**B**

In [ ]:
fig,ax = plt.subplots(figsize=(5,3))
# len(NLS)

for i in range(N_params):
    # solution_snapshot_f_nset = solution_snapshot_f[i*nt:(i+1)*nt,:]
    ax.plot(solution_snapshot_f_nset[i][:,41],solution_snapshot_f_nset[i][:,543])

plt.xlabel("Displacement")
plt.ylabel("Velocity")
plt.show()


In [ ]:
# Save the data set
solution_snapshot_f_test = solution_snapshot_f[]

os.chdir(notebook_dir)
np.save(f'data/snapshots_p_{len(param_cl[cluster])}params.npy',solution_snapshot_f)
np.save(f'data/params_{len(param_cl[cluster])}.npy',param_cl[cluster])

---

### Train-Test split

In [31]:
train_mask_t, _ = sobol_train_test_split(nt, train_percentage = 0.5)

In [32]:
solution_snapshot_ecsw=np.zeros((N_params,len(train_mask_t[train_mask_t==True]),solution_snapshot_f.shape[-1]))
solution_snapshot_ecsw_disp_svd = solution_snapshot_f[:nt,:][train_mask_t]

for i in range(1,N_params):
    solution_snapshot_ecsw[i] = solution_snapshot_f[i*nt:(i+1)*nt,:][train_mask_t]
    solution_snapshot_ecsw_da_svd = np.vstack([solution_snapshot_ecsw_disp_svd, solution_snapshot_ecsw[i]])

In [33]:
dim_da = int(solution_snapshot_ecsw_da_svd.shape[1]/2) # train only on disp_vel
NLS_wd_train_da = solution_snapshot_ecsw_disp_svd[:,:dim_da]

In [34]:
NLS_train_mean = np.mean(NLS_wd_train_da,axis=0)
NLS_train_ms = NLS_wd_train_da - NLS_train_mean

In [ ]:
NLS_train_mean.shape

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))

for i in range(NLS_train_ms.shape[0]):
    Plot = plot(d.xi[0][1:-1], NLS_train_ms[i][1:-1:2], ax=ax)
    Plot.line_()

plt.show()


#### Step 1: Perform SVD on the snapshots (calculate $\mathbb{V}(=\mathbb{W}$)):

In [ ]:
plt.rcParams['figure.dpi'] = 100
n_sel, U = svd_mode_selector(NLS_train_ms, tolerance=1e-15,modes=True)
V_sel = U[:, :n_sel]

In [ ]:
V_sel.shape

In [ ]:
fig, ax = plt.subplots()

for i in range(n_sel):
    Plot = plot(d.xi[0][1:-1], V_sel[1:-1:2,i], ax=ax)
    Plot.line_()

Plot.ax.set_xlabel('$x$')
Plot.ax.set_ylabel('$\psi(x)$')

plt.show()